# Бэггинг и случайный лес

Загрузите датасет digits с помощью функции load_digits из sklearn.datasets и подготовьте матрицу признаков X и ответы на обучающей выборке y (вам потребуются поля data и target в объекте, который возвращает load_digits).

Для оценки качества далее нужно будет использовать cross_val_score из sklearn.cross_validation с параметром cv=10. Эта функция реализует k-fold cross validation c k равным значению параметра cv. Мы предлагаем использовать k=10, чтобы полученные оценки качества имели небольшой разброс, и было проще проверить полученные ответы. На практике же часто хватает и k=5. Функция cross_val_score будет возвращать numpy.ndarray, в котором будет k чисел - качество в каждом из k экспериментов k-fold cross validation. Для получения среднего значения (которое и будет оценкой качества работы) вызовите метод .mean() у массива, который возвращает cross_val_score.

С небольшой вероятностью вы можете натолкнуться на случай, когда полученное вами качество в каком-то из пунктов не попадет в диапазон, заданный для правильных ответов - в этом случае попробуйте перезапустить ячейку с cross_val_score несколько раз и выбрать наиболее «типичное» значение. Если это не помогает, то где-то была допущена ошибка.

Если вам захочется ускорить вычисление cross_val_score - можете попробовать использовать параметр n_jobs, но будьте осторожны: в одной из старых версий sklearn была ошибка, которая приводила к неверному результату работы cross_val_score при задании n_jobs отличным от 1. Сейчас такой проблемы возникнуть не должно, но проверить, что все в порядке, не будет лишним.

In [70]:
from sklearn import datasets
import numpy as np
import pandas as pd
import math
from sklearn import model_selection, metrics
from sklearn.model_selection import cross_val_score as cv
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier


import warnings
warnings.filterwarnings('ignore')

In [12]:
X = pd.DataFrame(datasets.load_digits().data)
y = pd.DataFrame(datasets.load_digits().target)

## Задание 1

Создайте DecisionTreeClassifier с настройками по умолчанию и измерьте качество его работы с помощью cross_val_score. Эта величина и будет ответом в пункте 1.

In [35]:
classif = DecisionTreeClassifier()

cross_val = cv(classif,X, y,  cv = 10).mean()
print(cross_val)

0.8258131595282434


In [36]:
def write_answer(ans, text):
    with open(text, "w") as fout:
        fout.write(str(ans))

write_answer(cross_val, str('ans1.txt'))

## Задание 2

Воспользуйтесь BaggingClassifier из sklearn.ensemble, чтобы обучить бэггинг над DecisionTreeClassifier. Используйте в BaggingClassifier параметры по умолчанию, задав только количество деревьев равным 100.

Качество классификации новой модели - ответ в пункте 2. Обратите внимание, как соотносится качество работы композиции решающих деревьев с качеством работы одного решающего дерева.

In [41]:
bagging = BaggingClassifier(classif, n_estimators = 100)

cross_val_bag = cv(bagging,X, y,  cv = 10).mean()
print(cross_val_bag)
write_answer(cross_val_bag, str('ans2.txt'))

0.9231657355679703


## Задание 3

Теперь изучите параметры BaggingClassifier и выберите их такими, чтобы каждый базовый алгоритм обучался не на всех d признаках, а на корне из dслучайных признаков. Качество работы получившегося классификатора - ответ в пункте 3. Корень из числа признаков - часто используемая эвристика в задачах классификации, в задачах регрессии же часто берут число признаков, деленное на три. Но в общем случае ничто не мешает вам выбирать любое другое число случайных признаков.

In [49]:
bagging_1 = BaggingClassifier(classif, n_estimators = 100, max_features = int(math.sqrt(X.shape[1])))

cross_val_bag_1 = cv(bagging_1 ,X, y,  cv = 10).mean()
print(cross_val_bag_1)
write_answer(cross_val_bag_1, str('ans3.txt'))

0.9349006828057107


## Задание 4

Наконец, давайте попробуем выбирать случайные признаки не один раз на все дерево, а при построении каждой вершины дерева. Сделать это несложно: нужно убрать выбор случайного подмножества признаков в BaggingClassifier и добавить его в DecisionTreeClassifier. Какой параметр за это отвечает, можно понять из документации sklearn, либо просто попробовать угадать (скорее всего, у вас сразу получится). Попробуйте выбирать опять же корень из d признаков. Качество полученного классификатора на контрольной выборке и будет ответом в пункте 4.

In [50]:
classif_2 = DecisionTreeClassifier(max_features = int(math.sqrt(X.shape[1])))

bagging_2 = BaggingClassifier(classif_2, n_estimators = 100)

cross_val_bag_2 = cv(bagging_2 ,X, y,  cv = 10).mean()
print(cross_val_bag_2)
write_answer(cross_val_bag_2, str('ans4.txt'))

0.9482402234636871


## Задание 5

Полученный в пункте 4 классификатор - бэггинг на рандомизированных деревьях (в которых при построении каждой вершины выбирается случайное подмножество признаков и разбиение ищется только по ним). Это в точности соответствует алгоритму Random Forest, поэтому почему бы не сравнить качество работы классификатора с RandomForestClassifier из sklearn.ensemble. Сделайте это, а затем изучите, как качество классификации на данном датасете зависит от количества деревьев, количества признаков, выбираемых при построении каждой вершины дерева, а также ограничений на глубину дерева. Для наглядности лучше построить графики зависимости качества от значений параметров, но для сдачи задания это делать не обязательно.

На основе наблюдений выпишите через пробел номера правильных утверждений из приведенных ниже в порядке возрастания номера (это будет ответ в п.5)

1) Случайный лес сильно переобучается с ростом количества деревьев

2) При очень маленьком числе деревьев (5, 10, 15), случайный лес работает хуже, чем при большем числе деревьев

3) С ростом количества деревьев в случайном лесе, в какой-то момент деревьев становится достаточно для высокого качества классификации, а затем качество существенно не меняется.


In [84]:
classifier1 = RandomForestClassifier(n_estimators = 5)
classifier2 = RandomForestClassifier(n_estimators = 10)
classifier3 = RandomForestClassifier(n_estimators = 15)
classifier4 = RandomForestClassifier(n_estimators = 30)
classifier5 = RandomForestClassifier(n_estimators = 100)


score1 = cv(classifier1, X, y, cv=10).mean()
score2 = cv(classifier2, X, y, cv=10).mean()
score3 = cv(classifier3, X, y, cv=10).mean()
score4 = cv(classifier4, X, y, cv=10).mean()
score5 = cv(classifier5, X, y, cv=10).mean()

print(score1, score2, score3, score4, score5)

0.8775822470515209 0.9092706393544383 0.9315394165114835 0.9398975791433891 0.9482495344506517


### 2 3

4) При большом количестве признаков (для данного датасета - 40, 50) качество классификации становится хуже, чем при малом количестве признаков (5, 10). Это связано с тем, что чем меньше признаков выбирается в каждом узле, тем более различными получаются деревья (ведь деревья сильно неустойчивы к изменениям в обучающей выборке), и тем лучше работает их композиция.

5) При большом количестве признаков (40, 50, 60) качество классификации лучше, чем при малом количестве признаков (5, 10). Это связано с тем, что чем больше признаков - тем больше информации об объектах, а значит алгоритм может делать прогнозы более точно.

In [88]:
classifier1 = RandomForestClassifier(max_features = 5)
classifier2 = RandomForestClassifier(max_features = 10)
classifier3 = RandomForestClassifier(max_features = 40)
classifier4 = RandomForestClassifier(max_features = 50)
classifier5 = RandomForestClassifier(n_estimators = 60)


score1 = cv(classifier1, X, y, cv=10).mean()
score2 = cv(classifier2, X, y, cv=10).mean()
score3 = cv(classifier3, X, y, cv=10).mean()
score4 = cv(classifier4, X, y, cv=10).mean()
score5 = cv(classifier5, X, y, cv=10).mean()

print(score1, score2, score3, score4, score5)

0.9482278088144008 0.947135319677219 0.9370918684047176 0.9348758535071383 0.9399099937926753


6) При небольшой максимальной глубине деревьев (5-6) качество работы случайного леса намного лучше, чем без ограничения глубины, т.к. деревья получаются не переобученными. С ростом глубины деревьев качество ухудшается.

7) При небольшой максимальной глубине деревьев (5-6) качество работы случайного леса заметно хуже, чем без ограничений, т.к. деревья получаются недообученными. С ростом глубины качество сначала улучшается, а затем не меняется существенно, т.к. из-за усреднения прогнозов и различий деревьев их переобученность в бэггинге не сказывается на итоговом качестве (все деревья преобучены по-разному, и при усреднении они компенсируют переобученность друг-друга).

In [86]:
classifier1 = RandomForestClassifier(max_depth = 2)
classifier2 = RandomForestClassifier(max_depth = 4)
classifier3 = RandomForestClassifier(max_depth = 6)
classifier4 = RandomForestClassifier(max_depth = 10)
classifier5 = RandomForestClassifier()


score1 = cv(classifier1, X, y, cv=10).mean()
score2 = cv(classifier2, X, y, cv=10).mean()
score3 = cv(classifier3, X, y, cv=10).mean()
score4 = cv(classifier4, X, y, cv=10).mean()
score5 = cv(classifier5, X, y, cv=10).mean()

print(score1, score2, score3, score4, score5)

0.7946430788330229 0.8842302917442583 0.9298603351955308 0.9476784605834885 0.9510211049037863


In [89]:
ans = '2 3 4 7'
write_answer(ans, str('ans5.txt'))

2) При очень маленьком числе деревьев (5, 10, 15), случайный лес работает хуже, чем при большем числе деревьев

3) С ростом количества деревьев в случайном лесе, в какой-то момент деревьев становится достаточно для высокого качества классификации, а затем качество существенно не меняется.

4) При большом количестве признаков (для данного датасета - 40, 50) качество классификации становится хуже, чем при малом количестве признаков (5, 10). Это связано с тем, что чем меньше признаков выбирается в каждом узле, тем более различными получаются деревья (ведь деревья сильно неустойчивы к изменениям в обучающей выборке), и тем лучше работает их композиция.

7) При небольшой максимальной глубине деревьев (5-6) качество работы случайного леса заметно хуже, чем без ограничений, т.к. деревья получаются недообученными. С ростом глубины качество сначала улучшается, а затем не меняется существенно, т.к. из-за усреднения прогнозов и различий деревьев их переобученность в бэггинге не сказывается на итоговом качестве (все деревья преобучены по-разному, и при усреднении они компенсируют переобученность друг-друга).